In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from matplotlib.widgets import Slider
import xcoll.geometry.trajectories as tr
import xcoll.geometry.segments as seg
from xcoll.geometry.crossings.find_root import FindRoot
#%matplotlib ipympl

# %matplotlib tk

## other stuff

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example constants (replace with your actual values)
beta = 1          # v/c
pc = 1e9          # momentum*speed of light [eV/c]
X0 = 0.353#19.32        # radiation length [cm]

# Number of steps for MCS
N_steps = 100
# Total length of material [cm]
L_total = 1

def mcs_test(L_total, L_step, N_particles=1000):
    """
    Simulate transverse displacement and angle after MCS.
    Returns arrays of x positions for N_particles.
    """
    x_arr = np.zeros(N_particles)
    x_prime_arr = np.random.randn(N_particles)

    # Step length
    L = L_total / L_step
    steps=0
    while steps < L_total:
        # MCS angle RMS (radians)
        ran_1 = np.random.uniform(0,1,N_particles)
        ran_2 = np.random.uniform(0,1,N_particles)
        x0 = x_arr.copy()
        x0_prime = x_prime_arr.copy()
        theta0 = 13.6e6 / (beta * pc) * np.sqrt(L_total / X0) * (1 + 0.038 * np.log(L_total / (X0 * beta**2)))
        # Random angles for each particle
        dtheta = np.random.normal(0, theta0, N_particles)
        # Update x and x'
        x_arr += L_total * dtheta * ( ran_1/np.sqrt(12) + ran_2/2)
        x_prime_arr += dtheta * (ran_2)
        steps += L

    return x_arr, x_prime_arr

# Run the simulation
x_vals_50, x_prime_vals_50 = mcs_test(L_total= 100, L_step=1, N_particles=500000)
x_vals_100, x_prime_vals_100 = mcs_test(L_total= 100, L_step=10, N_particles=500000)
x_vals_200, x_prime_vals_200 = mcs_test(L_total= 100, L_step=100, N_particles=500000)
    




In [ ]:
import xcoll as xc

mat = xc.materials.MolybdenumGraphite

print(mat)

In [ ]:
# Plot histogram as probability density
plt.figure(figsize=(9,6))
plt.hist(x_prime_vals_50, bins=100, density=True, alpha=0.3, color='navy', label='1 step, 100 mm')
plt.hist(x_prime_vals_100, bins=100, density=True, alpha=0.5, color='blue', label='10 steps, 10 mm')
plt.hist(x_prime_vals_200, bins=100, density=True, alpha=0.6, color='indigo', label='100 steps, 1 mm')

plt.xlabel('Angular distribution [rad]', fontsize=25)
plt.ylabel('Probability density', fontsize=25)
plt.title('Transverse angular distribution after MCS', fontsize=25)
plt.legend(fontsize=15)
plt.xlim(-6, 6)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

In [ ]:

# mcs = tr.MultipleCoulombTrajectory(s0=11, x0=13, xp=np.tan(np.deg2rad(180.)), pc=1e9, beta=1.e9/np.sqrt(0.938e9**2 + 1.e9**2),
#                                      q=1, X0=0.0001, ran_1=0.6889723883213388 , ran_2=0.4150210773485264, l1=0,l2=1)


xp=np.tan(np.deg2rad(180.))
pc=1e9
beta=1.e9/np.sqrt(0.938e9**2 + 1.e9**2)
X0 = 0.0001
# ran_1=0.6889723883213388 
# ran_2=0.4150210773485264
ran_1 = np.random.rand()
ran_2 = np.random.rand()
L = 2
theta = 13.6 * 10**6 /(beta*pc) * np.sqrt(L/X0) * (1 + 0.038 * np.log(L/(X0*beta*beta)))

x = L * theta * ( ran_1/np.sqrt(12) + ran_2/2)
x_prime = theta * (ran_2)
def mcs_test(L, N_particles):
    accumulated_L = 0
    L = 100 / L
    x_arr = np.array([])
    x_prime_arr = np.array([])  
    x0 = 0
    x0_prime = 0
    i = 0
    while accumulated_L < 100:
        print(f"Step {i}: L = {L}, accumulated_L = {accumulated_L}")
        accumulated_L += L
        theta0= 13.6 * 10**6 /(beta*pc) * np.sqrt(L/X0) * (1 + 0.038 * np.log(L/(X0*beta*beta)))
        thetas = np.random.normal(0, theta0, N_particles)
        x = x0 + L * thetas * ( ran_1/np.sqrt(12) + ran_2/2)
        x_prime =  x0_prime + thetas* (ran_2)
        x_arr = np.append(x_arr, x)
        x_prime_arr = np.append(x_prime_arr, x_prime)
        x0= x
        x0_prime = x_prime
        i += 1
    return x_arr, x_prime_arr

x_arr, x_prime_arr = mcs_test(10)
plt.scatter(x_prime_arr)

In [ ]:
line= seg.LineSegment(s1=10.5, x1=15, s2=13, x2=13.5)  
drift= tr.DriftTrajectory(s0=10., x0=12.5, xp=np.deg2rad(50.0)) 
bezier = seg.BezierSegment(s1=11, x1=14.2, cs1=11.669, cx1=13., cs2=12., cx2=12., s2=12.669, x2=14)    
halfopen = seg.HalfOpenLineSegment(s1=9, x1=13, cos_t1=np.tan(np.deg2rad(44.5)))
mcs = tr.MultipleCoulombTrajectory(s0=11, x0=13, xp=np.tan(np.deg2rad(180.)), pc=1e9, beta=1.e9/np.sqrt(0.938e9**2 + 1.e9**2),
                                     q=1, X0=0.0001, ran_1=0.98 , ran_2=0.4150210773485264, l1=0,l2=1)


locseg = seg.segment.LocalSegment(bezier)
loctraj = tr.trajectory.LocalTrajectory(mcs)


line_box    = line.get_box()
drift_box   = drift.get_box(l1=0,l2=5)
# line_box1     = line1.get_box()
# drift_box1    = drift1.get_box(l1=0,l2=5)
# line_box2     = line2.get_box()
# drift_box2    = drift2.get_box(l1=0,l2=5)
# line_box3     = line3.get_box()
# drift_box3    = drift3.get_box(l1=0,l2=5)
bezier_box  = bezier.get_box(t1=0, t2=1)
mcs_box      = mcs.get_box(l1=0,l2=1)
#mcs_box.flag = -1.0

halfopen_box = halfopen.get_box(t1=0, t2=5)
#traj = tr.trajectory.LocalTrajectory.from_dict(drift.to_dict())

In [ ]:
ran_1=0.6889723883213388 , ran_2=0.4150210773485264

In [ ]:
plt.figure()
#mcs.plot()
bezier.plot()
#plt.figure(figsize=(8, 6))
plt.plot([line_box.s1, line_box.s2, line_box.s3, line_box.s4, line_box.s1],
         [line_box.x1, line_box.x2, line_box.x3, line_box.x4, line_box.x1], color='seagreen' , label='Line Segment')
plt.plot([drift_box.s1, drift_box.s2, drift_box.s3, drift_box.s4, drift_box.s1],
         [drift_box.x1, drift_box.x2, drift_box.x3, drift_box.x4, drift_box.x1], color='slateblue', label='Drift Trajectory')
plt.plot([mcs_box.s1, mcs_box.s2, mcs_box.s3, mcs_box.s4, mcs_box.s1],
         [mcs_box.x1, mcs_box.x2, mcs_box.x3, mcs_box.x4, mcs_box.x1], 'g-' )
plt.plot([halfopen_box.s1, halfopen_box.s2, halfopen_box.s3, halfopen_box.s4, halfopen_box.s1],
         [halfopen_box.x1, halfopen_box.x2, halfopen_box.x3, halfopen_box.x4, halfopen_box.x1], 'm-' )
plt.plot([bezier_box.s1, bezier_box.s2, bezier_box.s3, bezier_box.s4, bezier_box.s1],
         [bezier_box.x1, bezier_box.x2, bezier_box.x3, bezier_box.x4, bezier_box.x1], 'c-' )
plt.legend(fontsize=16)

plt.axis('equal')
plt.xlabel('s [m]',fontsize=15)
#plt.ylim(12.5,16)
plt.ylabel('x [m]',fontsize=15.)
plt.title('Intersection between Drift Trajectory and Line Segment',fontsize=15)
plt.axis()
plt.grid()
plt.show()

In [ ]:
analytical_t = np.array([]) 
analytical_l = np.array([])
cases = 100
drift_s0 = np.linspace(10, 11, cases)
#drift_x0 = np.linspace(12.5, 16.0, cases)
drift_xp = np.linspace(np.deg2rad(50), np.deg2rad(80.0), cases)
for i in range((cases)):
    line = seg.LineSegment(s1=10.5, x1=13.5, s2=14, x2=13.5)
    drift = tr.DriftTrajectory(s0=drift_s0[i], x0=12.5, xp=drift_xp[i])
    check = FindRoot()
    locseg= seg.segment.LocalSegment(line)
    loctraj= tr.trajectory.LocalTrajectory(drift)
    check.find_crossing(seg=locseg, traj=loctraj)
    analytical_l = np.append(analytical_l, check.solution_l[0])
    analytical_t = np.append(analytical_t, check.solution_t[0])

In [ ]:
def calculate_error(guess, numerical, analytical, relE, absE, guess_diff):
    print(f"numerical, analytical: {numerical}, {analytical}")
    print(f"relative error: {abs(numerical - analytical)/analytical}")
    relE = np.append(relE, abs(numerical - analytical)/analytical)
    absE = np.append(absE, abs(numerical - analytical))
    guess_diff = np.append(guess_diff, np.abs(guess - numerical))
    print(f"absolute error: {abs(numerical - analytical)}")
    print(f"guess - numerical: {abs(guess - numerical)}")
    return relE, absE, guess_diff


## Newton Convergence

In [ ]:


num_iterations = 7
# Initialize sum arrays for the two valid elements
res_t_finish = np.zeros(num_iterations)
res_l_finish = np.zeros(num_iterations)
delta_t_finish = np.zeros(num_iterations)
delta_l_finish = np.zeros(num_iterations)
time_cases = np.zeros(100)
delta_guess_sol_l_finish = np.zeros(1)
delta_guess_sol_t_finish = np.zeros(1)
cases=100
for i in range((cases)):
    check = FindRoot()
    #line_s1 = np.linspace(10, 11, cases)
    line_x1 = np.linspace(13.5, 15, cases)
    line = seg.LineSegment(s1=10.5, x1=line_x1[i], s2=14, x2=13.5)
    half_deg = np.linspace(37,44,cases)
    halfopen = seg.HalfOpenLineSegment(s1=10.5, x1=13, cos_t1=np.tan(np.deg2rad(half_deg[i])))
    bezier_x1 = np.linspace(13.2, 14.2, cases)
    bezier = seg.BezierSegment(s1=11, x1=bezier_x1[i], cs1=11.669, cx1=13., cs2=12., cx2=12., s2=12.669, x2=14)
    mcs_ran1 =np.linspace(0.6889723883213388, 0.98,cases)
    mcs = tr.MultipleCoulombTrajectory(s0=11, x0=13, xp=np.tan(np.deg2rad(180.)), pc=1e9, beta=1.e9/np.sqrt(0.938e9**2 + 1.e9**2),
                                     q=1, X0=0.0001, ran_1=mcs_ran1[i] , ran_2=0.4150210773485264, l1=0,l2=1)
    locseg= seg.segment.LocalSegment(halfopen)
    loctraj= tr.trajectory.LocalTrajectory(mcs)
    start_t = time.process_time()
    for k in range(200):
        check.find_crossing(seg=locseg, traj=loctraj)
    end_t = time.process_time()
    time_cases[i] = (end_t - start_t)/200

    res_t_data = check.res_t
    res_l_data = check.res_l
    delta_t_data = check.delta_t
    delta_l_data = check.delta_l 
    guess_l_data = check.guess_l
    guess_t_data = check.guess_t
    solution_l_data = check.solution_l
    solution_t_data = check.solution_t

    guess_t = np.array(check.guess_t)
    guess_l = np.array(check.guess_l)
    sol_t = np.array(check.solution_t)
    sol_l = np.array(check.solution_l)
    res_t = np.array(check.res_t)
    res_l = np.array(check.res_l)
    delta_t = np.array(check.delta_t)
    delta_l = np.array(check.delta_l)
    
    delta_guess_sol_l_data = np.abs(guess_l - sol_l)
    delta_guess_sol_t_data = np.abs(guess_t - sol_t)
    # Create a boolean mask for values below 1e21
    mask = (res_t < 1e20) & (res_l < 1e20) & (delta_t < 1e20) & (delta_l < 1e20)

    # Apply mask to keep only “valid” elements
    valid_res_t = res_t[:num_iterations]        # only the first two valid
    valid_res_l = res_l[:num_iterations]
    valid_delta_t = delta_t[:num_iterations]
    valid_delta_l = delta_l[:num_iterations]
    valid_delta_guess_sol_l = delta_guess_sol_l_data[:1]
    valid_delta_guess_sol_t = delta_guess_sol_t_data[:1]
    valid_res_t = np.where(valid_res_t < 1e20, valid_res_t, 1e-16)
    valid_res_l = np.where(valid_res_l < 1e20, valid_res_l, 1e-16)
    valid_delta_t = np.where(valid_delta_t < 1e20, valid_delta_t, 1e-16)
    valid_delta_l = np.where(valid_delta_l < 1e20, valid_delta_l, 1e-16)
    valid_delta_guess_sol_l = np.where(valid_delta_guess_sol_l < 1e20, valid_delta_guess_sol_l, 1e-16)
    valid_delta_guess_sol_t = np.where(valid_delta_guess_sol_t < 1e20, valid_delta_guess_sol_t, 1e-16)

    # Add to finish arrays
    res_t_finish += valid_res_t
    res_l_finish += valid_res_l
    delta_t_finish += valid_delta_t
    delta_l_finish += valid_delta_l
    delta_guess_sol_l_finish += valid_delta_guess_sol_l
    delta_guess_sol_t_finish += valid_delta_guess_sol_t
    #delta_time += end_t - start_t


res_t_mean = res_t_finish / cases
res_l_mean = res_l_finish / cases
delta_t_mean = delta_t_finish / cases
delta_l_mean = delta_l_finish / cases
delta_guess_sol_l_mean = delta_guess_sol_l_finish / cases
delta_guess_sol_t_mean = delta_guess_sol_t_finish / cases
#time_mean = delta_time / cases

    #relE_l, absE_l, guessdiff_l = calculate_error(check.guess_l[0],check.solution_l[0], analytical_l[i], relE_l, absE_l, guessdiff_l)
    #relE_t, absE_t, guessdiff_t = calculate_error(check.guess_t[0],check.solution_t[0], analytical_t[i], relE_t, absE_t, guessdiff_t)



In [ ]:
# Run once to check
check = FindRoot()
import time
time_start = time.time()
check.find_crossing(seg=locseg, traj=loctraj)
time_stop  = time.time()
print(f"Time to find crossing: {time_stop - time_start} seconds")
# The solutions are saved in the python class atm for testing. 
sol_t = check.solution_t
sol_l = check.solution_l
num = check.num_solutions
for i in range(len(check.guess_t)):
    if check.guess_t[i] < 1e21:
        print(f"Guess {i}: l = {check.guess_l[i]}, t = {check.guess_t[i]}")
for i in range(len(sol_t)):
    if sol_t[i] < 1e21:
        print(f"Solution {i}: l = {sol_l[i]}, t = {sol_t[i]}")

In [ ]:
# X-axis: iteration number
iterations = np.arange(num_iterations)
plt.figure(figsize=(6,4))


# Plot step changes (delta)
plt.semilogy(iterations, delta_t_mean, marker='o', 
             label='|Δt|')
plt.semilogy(iterations, delta_l_mean, marker='x', label='|Δl|')

# Plot residuals
plt.semilogy(iterations, res_t_mean, '--', label='|TS[0]|')
plt.semilogy(iterations, res_l_mean, '--', label='|TS[1]|')
plt.xlabel("Iteration",fontsize=15)
plt.ylabel("Error / Residual",fontsize=15)
plt.title("Newton Convergence",fontsize=15)
plt.legend()
plt.grid(True, which='both', linestyle='--', alpha=0.7)
plt.tight_layout()

plt.show()

In [ ]:
print(f"average res_t: {res_t_mean}")
print(f"average res_l: {res_l_mean}")
print(f"average delta_t: {delta_t_mean}")
print(f"average delta_l: {delta_l_mean}")
print(f"average delta_guess_sol_l: {delta_guess_sol_l_mean}")
print(f"time per case: {np.mean(time_cases)}")


In [ ]:
import xcoll as xc

# filepath_bezier = xc._pkg_root.parent / "results" / "mcs_bezier"
# filepath_halfopen = xc._pkg_root.parent / "results" / "mcs_half"
# filepath_line = xc._pkg_root.parent / "results" / "mcs_line"
filepath_bezier = xc._pkg_root.parent / "results" / "mcs_bezier_time"
filepath_halfopen = xc._pkg_root.parent / "results" / "mcs_half_time"
filepath_line = xc._pkg_root.parent / "results" / "mcs_line_time"

file = filepath_halfopen
nest_step = "2_4.npz"

In [ ]:

# # Save multiple arrays in one .npz file nest_step
#np.savez(file / nest_step,
         res_t=res_t_mean,
         res_l=res_l_mean,
         delta_t=delta_t_mean,
         delta_l=delta_l_mean,
         delta_guess_sol_l=delta_guess_sol_l_mean,
         delta_guess_sol_t=delta_guess_sol_t_mean,
         iterations=iterations,
         time=time_cases)
print(f"Saved benchmark data to {file / nest_step}")

In [ ]:

data_line = np.load(file / "2_4.npz")
time24_line = data_line["time"]
res_t_24_line = data_line["res_t"]
res_l_24_line = data_line["res_l"]
delta_t_24_line = data_line["delta_t"]
delta_l_24_line = data_line["delta_l"]
iterations_24_line = data_line["iterations"]

data_line = np.load(file / "2_6.npz")
time26_line = data_line["time"]
res_t_26_line = data_line["res_t"]
res_l_26_line = data_line["res_l"]
delta_t_26_line = data_line["delta_t"]
delta_l_26_line = data_line["delta_l"]
iterations_26_line = data_line["iterations"]

data_line = np.load(file / "2_8.npz")
time28_line = data_line["time"]
res_t_28_line = data_line["res_t"]
res_l_28_line = data_line["res_l"]
delta_t_28_line = data_line["delta_t"]
delta_l_28_line = data_line["delta_l"]
iterations_28_line = data_line["iterations"]

data_line = np.load(file / "4_8.npz")
time48_line = data_line["time"]
res_t_48_line = data_line["res_t"]
res_l_48_line = data_line["res_l"]
delta_t_48_line = data_line["delta_t"]
delta_l_48_line = data_line["delta_l"]
iterations_48_line = data_line["iterations"]

data_line = np.load(file / "4_6.npz")
time46_line = data_line["time"]
res_t_46_line = data_line["res_t"]
res_l_46_line = data_line["res_l"]
delta_t_46_line = data_line["delta_t"]
delta_l_46_line = data_line["delta_l"]
iterations_46_line = data_line["iterations"]
delta_guess_sol_l_46_line = data_line["delta_guess_sol_l"]
delta_guess_sol_t_46_line = data_line["delta_guess_sol_t"]

data_line = np.load(file / "4_4.npz")
time44_line = data_line["time"]
res_t_44_line = data_line["res_t"]
res_l_44_line = data_line["res_l"]
delta_t_44_line = data_line["delta_t"]
delta_l_44_line = data_line["delta_l"]
iterations_44_line = data_line["iterations"]
delta_guess_sol_l_44_line = data_line["delta_guess_sol_l"]
delta_guess_sol_t_44_line = data_line["delta_guess_sol_t"]

data_line = np.load(file / "6_4.npz")
time64_line = data_line["time"]
res_t_64_line = data_line["res_t"]
res_l_64_line = data_line["res_l"]
delta_t_64_line = data_line["delta_t"]
delta_l_64_line = data_line["delta_l"]
iterations_64_line = data_line["iterations"]
delta_guess_sol_l_64_line = data_line["delta_guess_sol_l"]
delta_guess_sol_t_64_line = data_line["delta_guess_sol_t"]

data_line = np.load(file / "6_6.npz")
time66_line = data_line["time"]
res_t_66_line = data_line["res_t"]
res_l_66_line = data_line["res_l"]
delta_t_66_line = data_line["delta_t"]
delta_l_66_line = data_line["delta_l"]
iterations_66_line = data_line["iterations"]
delta_guess_sol_l_66_line = data_line["delta_guess_sol_l"]
delta_guess_sol_t_66_line = data_line["delta_guess_sol_t"]

data_line = np.load(file / "6_8.npz")
time68_line = data_line["time"]
res_t_68_line = data_line["res_t"]
res_l_68_line = data_line["res_l"]
delta_t_68_line = data_line["delta_t"]
delta_l_68_line = data_line["delta_l"]
iterations_68_line = data_line["iterations"]
delta_guess_sol_l_68_line = data_line["delta_guess_sol_l"]
delta_guess_sol_t_68_line = data_line["delta_guess_sol_t"]

data_line = np.load(file / "8_4.npz")
time84_line = data_line["time"]
res_t_84_line = data_line["res_t"]
res_l_84_line = data_line["res_l"]
delta_t_84_line = data_line["delta_t"]
delta_l_84_line = data_line["delta_l"]
iterations_84_line = data_line["iterations"]

data_line = np.load(file / "8_6.npz")
time86_line = data_line["time"]
res_t_86_line = data_line["res_t"]
res_l_86_line = data_line["res_l"]
delta_t_86_line = data_line["delta_t"]
delta_l_86_line = data_line["delta_l"]
iterations_86_line = data_line["iterations"]

data_line = np.load(file / "8_8.npz")
time88_line = data_line["time"]
res_t_88_line = data_line["res_t"]
res_l_88_line = data_line["res_l"]
delta_t_88_line = data_line["delta_t"]
delta_l_88_line = data_line["delta_l"]
iterations_88_line = data_line["iterations"]
delta_guess_sol_l_88_line = data_line["delta_guess_sol_l"]
delta_guess_sol_t_88_line = data_line["delta_guess_sol_t"]

In [ ]:

data_bezier = np.load(file / "2_4.npz")
time24_bezier = data_bezier["time"]
res_t_24_bezier = data_bezier["res_t"]
res_l_24_bezier = data_bezier["res_l"]
delta_t_24_bezier = data_bezier["delta_t"]
delta_l_24_bezier = data_bezier["delta_l"]
iterations_24_bezier = data_bezier["iterations"]

data_bezier = np.load(file / "2_6.npz")
time26_bezier = data_bezier["time"]
res_t_26_bezier = data_bezier["res_t"]
res_l_26_bezier = data_bezier["res_l"]
delta_t_26_bezier = data_bezier["delta_t"]
delta_l_26_bezier = data_bezier["delta_l"]
iterations_26_bezier = data_bezier["iterations"]

data_bezier = np.load(file / "2_8.npz")
time28_bezier = data_bezier["time"]
res_t_28_bezier = data_bezier["res_t"]
res_l_28_bezier = data_bezier["res_l"]
delta_t_28_bezier = data_bezier["delta_t"]
delta_l_28_bezier = data_bezier["delta_l"]
iterations_28_bezier = data_bezier["iterations"]

data_bezier = np.load(file / "4_8.npz")
time48_bezier = data_bezier["time"]
res_t_48_bezier = data_bezier["res_t"]
res_l_48_bezier = data_bezier["res_l"]
delta_t_48_bezier = data_bezier["delta_t"]
delta_l_48_bezier = data_bezier["delta_l"]
iterations_48_bezier = data_bezier["iterations"]

data_bezier = np.load(file / "4_6.npz")
time46_bezier = data_bezier["time"]
res_t_46_bezier = data_bezier["res_t"]
res_l_46_bezier = data_bezier["res_l"]
delta_t_46_bezier = data_bezier["delta_t"]
delta_l_46_bezier = data_bezier["delta_l"]
iterations_46_bezier = data_bezier["iterations"]
delta_guess_sol_l_46_bezier = data_bezier["delta_guess_sol_l"]
delta_guess_sol_t_46_bezier = data_bezier["delta_guess_sol_t"]

data_bezier = np.load(file / "4_4.npz")
time44_bezier = data_bezier["time"]
res_t_44_bezier = data_bezier["res_t"]
res_l_44_bezier = data_bezier["res_l"]
delta_t_44_bezier = data_bezier["delta_t"]
delta_l_44_bezier = data_bezier["delta_l"]
iterations_44_bezier = data_bezier["iterations"]
delta_guess_sol_l_44_bezier = data_bezier["delta_guess_sol_l"]
delta_guess_sol_t_44_bezier = data_bezier["delta_guess_sol_t"]

data_bezier = np.load(file / "6_4.npz")
time64_bezier = data_bezier["time"]
res_t_64_bezier = data_bezier["res_t"]
res_l_64_bezier = data_bezier["res_l"]
delta_t_64_bezier = data_bezier["delta_t"]
delta_l_64_bezier = data_bezier["delta_l"]
iterations_64_bezier = data_bezier["iterations"]
delta_guess_sol_l_64_bezier = data_bezier["delta_guess_sol_l"]
delta_guess_sol_t_64_bezier = data_bezier["delta_guess_sol_t"]

data_bezier = np.load(file / "6_6.npz")
time66_bezier = data_bezier["time"]
res_t_66_bezier = data_bezier["res_t"]
res_l_66_bezier = data_bezier["res_l"]
delta_t_66_bezier = data_bezier["delta_t"]
delta_l_66_bezier = data_bezier["delta_l"]
iterations_66_bezier = data_bezier["iterations"]
delta_guess_sol_l_66_bezier = data_bezier["delta_guess_sol_l"]
delta_guess_sol_t_66_bezier = data_bezier["delta_guess_sol_t"]

data_bezier = np.load(file / "6_8.npz")
time68_bezier = data_bezier["time"]
res_t_68_bezier = data_bezier["res_t"]
res_l_68_bezier = data_bezier["res_l"]
delta_t_68_bezier = data_bezier["delta_t"]
delta_l_68_bezier = data_bezier["delta_l"]
iterations_68_bezier = data_bezier["iterations"]
delta_guess_sol_l_68_bezier = data_bezier["delta_guess_sol_l"]
delta_guess_sol_t_68_bezier = data_bezier["delta_guess_sol_t"]

data_bezier= np.load(file / "8_4.npz")
time84_bezier = data_bezier["time"]
res_t_84_bezier = data_bezier["res_t"]
res_l_84_bezier = data_bezier["res_l"]
delta_t_84_bezier = data_bezier["delta_t"]
delta_l_84_bezier = data_bezier["delta_l"]
iterations_84_bezier = data_bezier["iterations"]

data_bezier = np.load(file / "8_6.npz")
time86_bezier = data_bezier["time"]
res_t_86_bezier = data_bezier["res_t"]
res_l_86_bezier = data_bezier["res_l"]
delta_t_86_bezier = data_bezier["delta_t"]
delta_l_86_bezier = data_bezier["delta_l"]
iterations_86_bezier = data_bezier["iterations"]

data_bezier = np.load(file / "8_8.npz")
time88_bezier = data_bezier["time"]
res_t_88_bezier = data_bezier["res_t"]
res_l_88_bezier = data_bezier["res_l"]
delta_t_88_bezier = data_bezier["delta_t"]
delta_l_88_bezier = data_bezier["delta_l"]
iterations_88_bezier = data_bezier["iterations"]
delta_guess_sol_l_88_bezier = data_bezier["delta_guess_sol_l"]
delta_guess_sol_t_88_bezier = data_bezier["delta_guess_sol_t"]

In [ ]:

data_hline = np.load(file / "2_4.npz")
time24_hline = data_hline["time"]
res_t_24_hline = data_hline["res_t"]
res_l_24_hline = data_hline["res_l"]
delta_t_24_hline = data_hline["delta_t"]
delta_l_24_hline = data_hline["delta_l"]
iterations_24_hline = data_hline["iterations"]

data_hline = np.load(file / "2_6.npz")
time26_hline = data_hline["time"]
res_t_26_hline = data_hline["res_t"]
res_l_26_hline = data_hline["res_l"]
delta_t_26_hline = data_hline["delta_t"]
delta_l_26_hline = data_hline["delta_l"]
iterations_26_hline = data_hline["iterations"]

data_hline = np.load(file / "2_8.npz")
time28_hline = data_hline["time"]
res_t_28_hline = data_hline["res_t"]
res_l_28_hline = data_hline["res_l"]
delta_t_28_hline = data_hline["delta_t"]
delta_l_28_hline = data_hline["delta_l"]
iterations_28_hline = data_hline["iterations"]

data_hline = np.load(file / "4_8.npz")
time48_hline = data_hline["time"]
res_t_48_hline = data_hline["res_t"]
res_l_48_hline = data_hline["res_l"]
delta_t_48_hline = data_hline["delta_t"]
delta_l_48_hline = data_hline["delta_l"]
iterations_48_hline = data_hline["iterations"]

data_hline = np.load(file / "4_6.npz")
time46_hline = data_hline["time"]
res_t_46_hline = data_hline["res_t"]
res_l_46_hline = data_hline["res_l"]
delta_t_46_hline = data_hline["delta_t"]
delta_l_46_hline = data_hline["delta_l"]
iterations_46_hline = data_hline["iterations"]
delta_guess_sol_l_46_hline = data_hline["delta_guess_sol_l"]
delta_guess_sol_t_46_hline = data_hline["delta_guess_sol_t"]

data_hline = np.load(file / "4_4.npz")
time44_hline = data_hline["time"]
res_t_44_hline = data_hline["res_t"]
res_l_44_hline = data_hline["res_l"]
delta_t_44_hline = data_hline["delta_t"]
delta_l_44_hline = data_hline["delta_l"]
iterations_44_hline = data_hline["iterations"]
delta_guess_sol_l_44_hline = data_hline["delta_guess_sol_l"]
delta_guess_sol_t_44_hline = data_hline["delta_guess_sol_t"]

data_hline = np.load(file / "6_4.npz")
time64_hline = data_hline["time"]
res_t_64_hline = data_hline["res_t"]
res_l_64_hline = data_hline["res_l"]
delta_t_64_hline = data_hline["delta_t"]
delta_l_64_hline = data_hline["delta_l"]
iterations_64_hline = data_hline["iterations"]
delta_guess_sol_l_64_hline = data_hline["delta_guess_sol_l"]
delta_guess_sol_t_64_hline = data_hline["delta_guess_sol_t"]

data_hline = np.load(file / "6_6.npz")
time66_hline = data_hline["time"]
res_t_66_hline = data_hline["res_t"]
res_l_66_hline = data_hline["res_l"]
delta_t_66_hline = data_hline["delta_t"]
delta_l_66_hline = data_hline["delta_l"]
iterations_66_hline = data_hline["iterations"]
delta_guess_sol_l_66_hline = data_hline["delta_guess_sol_l"]
delta_guess_sol_t_66_hline = data_hline["delta_guess_sol_t"]

data_hline = np.load(file / "6_8.npz")
time68_hline = data_hline["time"]
res_t_68_hline = data_hline["res_t"]
res_l_68_hline = data_hline["res_l"]
delta_t_68_hline = data_hline["delta_t"]
delta_l_68_hline = data_hline["delta_l"]
iterations_68_hline = data_hline["iterations"]
delta_guess_sol_l_68_hline = data_hline["delta_guess_sol_l"]
delta_guess_sol_t_68_hline = data_hline["delta_guess_sol_t"]

data_hline = np.load(file / "8_4.npz")
time84_hline = data_hline["time"]
res_t_84_hline = data_hline["res_t"]
res_l_84_hline = data_hline["res_l"]
delta_t_84_hline = data_hline["delta_t"]
delta_l_84_hline = data_hline["delta_l"]
iterations_84_hline = data_hline["iterations"]

data_hline = np.load(file / "8_6.npz")
time86_hline = data_hline["time"]
res_t_86_hline = data_hline["res_t"]
res_l_86_hline = data_hline["res_l"]
delta_t_86_hline = data_hline["delta_t"]
delta_l_86_hline = data_hline["delta_l"]
iterations_86_hline = data_hline["iterations"]

data_hline = np.load(file / "8_8.npz")
time88_hline = data_hline["time"]
res_t_88_hline = data_hline["res_t"]
res_l_88_hline = data_hline["res_l"]
delta_t_88_hline = data_hline["delta_t"]
delta_l_88_hline = data_hline["delta_l"]
iterations_88_hline = data_hline["iterations"]
delta_guess_sol_l_88_hline = data_hline["delta_guess_sol_l"]
delta_guess_sol_t_88_hline = data_hline["delta_guess_sol_t"]

In [ ]:
# # Compute means and stds
time_arrays_line = [time24_line, time26_line, time28_line,
                    time44_line, time46_line, time48_line,
                    time64_line, time66_line, time68_line,
                    time84_line, time86_line, time88_line]
means_line = [np.mean(arr) for arr in time_arrays_line]
stds_line  = [np.std(arr)  for arr in time_arrays_line]

time_arrays_bezier = [time24_bezier, time26_bezier, time28_bezier,
                      time44_bezier, time46_bezier, time48_bezier,
                      time64_bezier, time66_bezier, time68_bezier,
                      time84_bezier, time86_bezier, time88_bezier]
means_bezier = [np.mean(arr) for arr in time_arrays_bezier]
stds_bezier  = [np.std(arr)  for arr in time_arrays_bezier]

time_arrays_halfopen = [time24_hline, time26_hline, time28_hline,
                        time44_hline, time46_hline, time48_hline,
                        time64_hline, time66_hline, time68_hline,
                        time84_hline, time86_hline, time88_hline]
means_halfopen = [np.mean(arr) for arr in time_arrays_halfopen]
stds_halfopen  = [np.std(arr)  for arr in time_arrays_halfopen]

labels = ["(2,4)","(2,6)","(2,8)",
          "(4,4)","(4,6)","(4,8)",
          "(6,4)","(6,6)","(6,8)",
          "(8,4)","(8,6)","(8,8)"]

# # Plot
# plt.figure(figsize=(11,8))
# plt.plot(labels, means_line, marker='o', label="MCS and Line segment", color="dodgerblue")
# plt.plot(labels, means_bezier, marker='s', label="MCS and Bézier segment",color="deeppink")
# # plt.plot(labels, means_halfopen, marker='^', label="MCS and Half-open segment", color="navy")

# # plt.xticks(rotation=45)
# plt.grid(True, which='both', linestyle='--', alpha=0.7)
# plt.legend(fontsize=20)
# vline_labels = ["(2,8)", "(4,8)", "(6,8)", "(8,8)"]
# vline_indices = [labels.index(lbl) for lbl in vline_labels]
# for idx in vline_indices:
#     plt.axvline(x=idx, color='slateblue', linestyle='--', linewidth=1)

# plt.title("Average Runtime for Newton-Raphson method", fontsize=30, pad=20)
# plt.ylabel("Time (seconds)", fontsize=25, labelpad=20)
# plt.xlabel("(Nesting lvl, Slicing No.)", fontsize=25, labelpad=20)
# plt.gca().xaxis.set_tick_params(labelsize=20)
# plt.gca().yaxis.set_tick_params(labelsize=20)
# plt.tight_layout()
# plt.show()
# Filter indices where label ends with ",8)"
selected_indices = [i for i, lbl in enumerate(labels) if lbl.endswith(",8)")]

# Extract filtered labels and values
labels_8 = [labels[i] for i in selected_indices]
means_line_8 = [means_line[i] for i in selected_indices]
means_bezier_8 = [means_bezier[i] for i in selected_indices]
means_halfopen_8 = [means_halfopen[i] for i in selected_indices]


fig, axes = plt.subplots(1, 2, figsize=(28, 14), sharex=True)
fig.suptitle("Average Runtime for Finding Intersections", fontsize=55, y=1.02)
# -------------------------
# Left subplot: Line + Bézier
# -------------------------
ax = axes[0]
ax.plot(labels, means_line, marker='o', linewidth=2, label="MCS and Line Segment", color="dodgerblue")
ax.plot(labels, means_bezier, marker='s', linewidth=2, label="MCS and Bézier Segment", color="deeppink")
ax.fill_between(labels, 
                np.array(means_bezier) - np.array(stds_bezier), 
                np.array(means_bezier) + np.array(stds_bezier), 
                color='deeppink', alpha=0.1, label="Bézier Segment Std. Dev.")

ax.fill_between(labels, 
                np.array(means_line) - np.array(stds_line), 
                np.array(means_line) + np.array(stds_line), 
                color='dodgerblue', alpha=0.14, label="Line Segment Std. Dev.")
vline_labels = ["(2,8)", "(4,8)", "(6,8)", "(8,8)"]
vline_indices = [labels.index(lbl) for lbl in vline_labels]
for idx in vline_indices:
    ax.axvline(x=idx, color='slateblue', linestyle='--', linewidth=1.5)

ax.set_title("Line and Bézier Segments", fontsize=40, pad=15)
ax.set_xlabel("(Nesting lvl, Slicing No.)", fontsize=40, labelpad=20)
ax.set_ylabel("Avg. Time (s)", fontsize=40, labelpad=20)
ax.grid(True, which='both', linestyle='--', alpha=0.7)
ax.legend(fontsize=25)
ax.tick_params(axis='x', labelrotation=45, labelsize=30)
ax.tick_params(axis='y', labelsize=30)
# -------------------------
# Right subplot: Line + Bézier + Half-open
# -------------------------
ax = axes[1]
ax.plot(labels, means_line, marker='o', linewidth=2, linestyle='-',label="MCS and Line Segment", color="dodgerblue")
ax.plot(labels, means_bezier, marker='s', linewidth=2, label="MCS and Bézier Segment", color="deeppink")
ax.plot(labels, means_halfopen, marker='^', linewidth=2, label="MCS and Half-Open Segment", color="navy")
for idx in vline_indices:
    ax.axvline(x=idx, color='slateblue', linestyle='--', linewidth=1.5)
ax.fill_between(labels, 
                np.array(means_halfopen) - np.array(stds_halfopen), 
                np.array(means_halfopen) + np.array(stds_halfopen), 
                color='blue', alpha=0.1, label="Half-Open Line Segment Std. Dev.")
ax.fill_between(labels, 
                np.array(means_bezier) - np.array(stds_bezier), 
                np.array(means_bezier) + np.array(stds_bezier), 
                color='deeppink', alpha=0.1, label="Bézier Segment Std. Dev.")
ax.fill_between(labels, 
                np.array(means_line) - np.array(stds_line), 
                np.array(means_line) + np.array(stds_line), 
                color='dodgerblue', alpha=0.1, label="Line Segment Std. Dev.")
ax.set_title("Line, Bézier, and Half-Open Line Segments", fontsize=40, pad=15)
ax.set_xlabel("(Nesting lvl, Slicing No.)", fontsize=40, labelpad=20)
ax.grid(True, which='both', linestyle='--', alpha=0.7)
ax.legend(fontsize=25)
ax.tick_params(axis='x', labelrotation=45, labelsize=30)
ax.tick_params(axis='y', labelsize=30)

plt.tight_layout()
plt.show()



In [ ]:

# ---------------------------
# Plot
# ---------------------------
plt.figure(figsize=(9,7))


# Step changes (delta)

plt.semilogy(iterations_24, res_t_24, marker='o', linestyle='-', label='|Δt|_4',color="orchid", alpha=1)
plt.semilogy(iterations_24, res_l_24, marker='^', linestyle='-', label='|Δl|_4',color="orchid", alpha=1)

plt.semilogy(iterations_26, res_t_26, marker='o', linestyle='--', label='|Δt|_4',color="slateblue", alpha=0.5)
plt.semilogy(iterations_26, res_l_26, marker='^', linestyle='--', label='|Δl|_4',color="slateblue", alpha=0.5)

plt.semilogy(iterations_28, res_t_28, marker='o', linestyle='--', label='|Δt|_4',color="cornflowerblue", alpha=0.5)
plt.semilogy(iterations_28, res_l_28, marker='^', linestyle='--', label='|Δl|_4',color="cornflowerblue", alpha=0.5)

# Step changes (res, alpha=0.5)
plt.semilogy(iterations_44, res_t_44, marker='o', linestyle='-', label='|Δt|_4',color="dodgerblue", alpha=1)
plt.semilogy(iterations_44, res_l_44, marker='^', linestyle='-', label='|Δl|_4',color="dodgerblue", alpha=1)

# Step changes (res, alpha=0.5)
plt.semilogy(iterations_46, res_t_46, marker='o', linestyle='--', label='|Δt|_6',color='mediumvioletred', alpha=0.5)
plt.semilogy(iterations_46, res_l_46, marker='^', linestyle='--', label='|Δl|_6',color='mediumvioletred', alpha=0.5)

plt.semilogy(iterations_48, res_t_48, marker='o', linestyle='--', label='|Δt|', color='orange', alpha=0.5)
plt.semilogy(iterations_48, res_l_48, marker='^', linestyle='--', label='|Δl|', color='orange', alpha=0.5)

plt.semilogy(iterations_64, res_t_64, marker='o', linestyle='--', label='|Δt|_6',color='purple', alpha=0.5)
plt.semilogy(iterations_64, res_l_64, marker='^', linestyle='--', label='|Δl|_6',color='purple', alpha=0.5)

plt.semilogy(iterations_66, res_t_66, marker='o', linestyle='-', label='|Δt|_6',color='green', alpha=1)
plt.semilogy(iterations_66, res_l_66, marker='^', linestyle='-', label='|Δl|_6',color='green', alpha=1)

plt.semilogy(iterations_68, res_t_68, marker='o', linestyle='--', label='|Δt|_6',color='red', alpha=0.5)
plt.semilogy(iterations_68, res_l_68, marker='^', linestyle='--', label='|Δl|_6',color='red', alpha=0.5)

plt.semilogy(iterations_84, res_t_84, marker='o', linestyle='--', label='|Δt|_6',color='midnightblue', alpha=0.5)
plt.semilogy(iterations_84, res_l_84, marker='^', linestyle='--', label='|Δl|_6',color='midnightblue', alpha=0.5)

plt.semilogy(iterations_86, res_t_86, marker='o', linestyle='--', label='|Δt|_6',color='darkorange', alpha=0.5)
plt.semilogy(iterations_86, res_l_86, marker='^', linestyle='--', label='|Δl|_6',color='darkorange', alpha=0.5)

plt.semilogy(iterations_88, res_t_88, marker='o', linestyle='--', label='|Δt|_6',color='maroon', alpha=0.5)
plt.semilogy(iterations_88, res_l_88, marker='^', linestyle='--', label='|Δl|_6',color='maroon', alpha=0.5)

plt.xlabel("Iterations", fontsize=27)
#plt.ylabel("Error / Residual", fontsize=25)
plt.title("Convergence for Bèzier Segment", fontsize=25)
#plt.legend()
h1 = plt.Line2D([], [], marker='o', linestyle='-', color='black', label='|Δt|')
h2 = plt.Line2D([], [], marker='^', linestyle='-', color='black', label=r'|Δ$\lambda$|')
h15 = plt.Line2D([], [], marker='o',linestyle='--', color='black', label=r'F[0]')
h16 = plt.Line2D([], [], marker='^',linestyle='--', color='black', label=r'F[1]')
h12 = plt.Line2D([], [], linestyle='-', color='orchid', label='(2,4)')
h13 = plt.Line2D([], [], linestyle='-', color='slateblue', label='(2,6)')
h14 = plt.Line2D([], [], linestyle='-', color='cornflowerblue', label='(2,8)')
h3 = plt.Line2D([], [], linestyle='-', color='dodgerblue', label='(4,4)')
h4 = plt.Line2D([], [], linestyle='-', color='mediumvioletred', label='(4,6)')
# h5 = plt.Line2D([], [], linestyle='-', color='orange', label='(4,8)')
h6 = plt.Line2D([], [], linestyle='-', color='purple', label='(6,4), (4,8)')
h7 = plt.Line2D([], [], linestyle='-', color='green', label='(6,6)')
h8 = plt.Line2D([], [], linestyle='-', color='red', label='(6,8)')
h9 = plt.Line2D([], [], linestyle='-', color='midnightblue', label='(8,4)')
h10 = plt.Line2D([], [], linestyle='-', color='darkorange', label='(8,6)')
h11 = plt.Line2D([], [], linestyle='-', color='maroon', label='(8,8)')
# plt.legend(handles=[h1, h2,h15,h16,h12,h13,h14, h3, h4, h6, h7, h8, h9, h10, h11], fontsize=14,ncol=2)

handles = [h1, h2, h15, h16, h12, h13, h14, h3, h4, h6, h7, h8, h9, h10, h11]

from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple

# Split handles into two groups
col1 = [h1, h2, h15, h16, h12, h13]  # first 6 entries
col2 = [h14, h3, h4, h6, h7, h8, h9, h10, h11]  # rest

# Combine handles into tuples for legend
all_handles = [(h,) for h in col1] + [(h,) for h in col2]
all_labels = [h.get_label() for h in col1] + [h.get_label() for h in col2]

# plt.axvline(x=2, color='red', linestyle='--', linewidth=1)
# plt.axvline(x=4, color='gray', linestyle='--', linewidth=1)
# plt.axvline(x=3, color='gray', linestyle='--', linewidth=1)
# Create legend with 2 columns
plt.legend(all_handles, all_labels, fontsize=12, ncol=2, loc='upper right', handlelength=2)
plt.grid(True, which='both', linestyle='--', alpha=0.7)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.minorticks_on()
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Define the figure with 2x2 subplots
fig, axes = plt.subplots(2, 2, figsize=(14,10), sharey=True,sharex=True)
axes = axes.flatten()
step_colors = {4: 'dodgerblue', 6: 'navy', 8: 'deeppink'}

# Group the data by “first number” in the tuple
groups = {
    '2': [(iterations_24, delta_t_24, delta_l_24, 4, '(2,4)', 1., '-'),
            (iterations_26, delta_t_26, delta_l_26, 6, '(2,6)', 0.5, '--'),
            (iterations_28, delta_t_28, delta_l_28, 8, '(2,8)', 0.5, '--')],
    
    '4': [(iterations_44, delta_t_44, delta_l_44, 4, '(4,4)',0.5, '--'),
            (iterations_46, delta_t_46, delta_l_46, 6, '(4,6)', 1, '-'),
            (iterations_48, delta_t_48, delta_l_48, 8, '(4,8)', 0.5, '--')],
    
    '6': [(iterations_64, delta_t_64, delta_l_64, 4, '(6,4)', 0.5, '--'),
            (iterations_66, delta_t_66, delta_l_66, 6, '(6,6)', 1.0, '-'),
            (iterations_68, delta_t_68, delta_l_68, 8, '(6,8)', 0.5, '--')],
    
    '8': [(iterations_84, delta_t_84, delta_l_84, 4, '(8,4)',0.5, '--'),
            (iterations_86, delta_t_86, delta_l_86, 6, '(8,6)', 0.5, '--'),
            (iterations_88, delta_t_88, delta_l_88, 8, '(8,8)', 0.5, '--')]
}

# Plot each group in a subplot
for ax, (group_name, curves) in zip(axes, groups.items()):
    for iters, res_t, res_l, step, label, alpha, linestyle in curves:
        # Plot Δt and Δl together with same marker
        color = step_colors[step]
        ax.semilogy(iters, res_t, marker='o', linestyle=linestyle, color=color, alpha=alpha, linewidth=2)
        ax.semilogy(iters, res_l, marker='^', linestyle=linestyle, color=color, alpha=alpha, linewidth=2)
    
    ax.set_title(f'Nesting level {group_name}', fontsize=26)
    ax.set_xlabel('Iterations', fontsize=24)
    ax.grid(True, which='both', linestyle='--', alpha=0.6)
    ax.tick_params(axis='both', labelsize=20)

# Set shared ylabel
axes[0].set_ylabel('Δ values', fontsize=25)
axes[2].set_ylabel('Δ values', fontsize=25)

# Create a single legend for all subplots
handles = np.array([plt.Line2D([], [], marker='o', linestyle='-', color='black', label='|Δt|'),
                    plt.Line2D([], [], marker='^', linestyle='-', color='black', label=r'|Δ$\lambda$|')])
labels = []
fig.suptitle("|Δt| and |Δλ| vs Iterations for Bèzier Segment", fontsize=30)
handles = np.append(handles, [plt.Line2D([], [], color=color, marker='o', linestyle='-') for color in step_colors.values()])
labels = ['|Δt|', r"|Δ$\lambda$|", 'Slicing number 4', 'Slicing number 6', 'Slicing number 8']
fig.legend(handles, labels, fontsize=18, loc='upper center', ncol=5, bbox_to_anchor=(0.53, 0.93))
plt.tight_layout(rect=[0,0,1,0.93])
plt.grid(True, which='both', linestyle='--', alpha=0.9)
plt.minorticks_on()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Define the figure with 2x2 subplots
fig, axes = plt.subplots(2, 2, figsize=(14,10), sharey=True,sharex=True)
axes = axes.flatten()
step_colors = {4: 'dodgerblue', 6: 'navy', 8: 'deeppink'}

# Group the data by “first number” in the tuple
groups = {
    '2': [(iterations_24, res_t_24, res_l_24, 4, '(2,4)', 0.5, '--'),
            (iterations_26, res_t_26, res_l_26, 6, '(2,6)', 0.5, '--'),
            (iterations_28, res_t_28, res_l_28, 8, '(2,8)', 1, '-')],
    
    '4': [(iterations_44, res_t_44, res_l_44, 4, '(4,4)',0.5, '--'),
            (iterations_46, res_t_46, res_l_46, 6, '(4,6)', 1, '-'),
            (iterations_48, res_t_48, res_l_48, 8, '(4,8)', 0.5, '--')],
    
    '6': [(iterations_64, res_t_64, res_l_64, 4, '(6,4)', 0.5, '--'),
            (iterations_66, res_t_66, res_l_66, 6, '(6,6)', 1.0, '-'),
            (iterations_68, res_t_68, res_l_68, 8, '(6,8)', 0.5, '--')],
    
    '8': [(iterations_84, res_t_84, res_l_84, 4, '(8,4)',0.5, '--'),
            (iterations_86, res_t_86, res_l_86, 6, '(8,6)', 0.5, '--'),
            (iterations_88, res_t_88, res_l_88, 8, '(8,8)', 0.5, '--')]
}

# Plot each group in a subplot
for ax, (group_name, curves) in zip(axes, groups.items()):
    for iters, res_t, res_l, step, label, alpha, linestyle in curves:
        # Plot Δt and Δl together with same marker
        color = step_colors[step]
        ax.semilogy(iters, res_t, marker='o', linestyle=linestyle, color=color, alpha=alpha, linewidth=2)
        ax.semilogy(iters, res_l, marker='^', linestyle=linestyle, color=color, alpha=alpha, linewidth=2)
    
    ax.set_title(f'Nesting level {group_name}', fontsize=26)
    ax.set_xlabel('Iterations', fontsize=24)
    ax.grid(True, which='both', linestyle='--', alpha=0.6)
    ax.tick_params(axis='both', labelsize=20)

# Set shared ylabel
axes[0].set_ylabel('Residuals', fontsize=25)
axes[2].set_ylabel('Residuals', fontsize=25)

# Create a single legend for all subplots
handles = np.array([plt.Line2D([], [], marker='o', linestyle='-', color='black', label='t residual'),
                    plt.Line2D([], [], marker='^', linestyle='-', color='black', label=r'$\lambda$ residual')])
labels = []
fig.suptitle('Residuals vs iterations for Half-Open Line Segment', fontsize=30)
handles = np.append(handles, [plt.Line2D([], [], color=color, marker='o', linestyle='-') for color in step_colors.values()])
labels = ['t residual', r"$\lambda$ residual", 'Slicing number 4', 'Slicing number 6', 'Slicing number 8']
fig.legend(handles, labels, fontsize=16.2, loc='upper center', ncol=5, bbox_to_anchor=(0.53, 0.93))
plt.tight_layout(rect=[0,0,1,0.93])
plt.grid(True, which='both', linestyle='--', alpha=0.9)
plt.minorticks_on()
plt.show()


In [ ]:

# ---------------------------
# Plot
# ---------------------------
plt.figure(figsize=(9,7))


# Step changes (delta)

plt.semilogy(iterations_24, delta_t_24, marker='o', label='|Δt|_4',color="orchid")
plt.semilogy(iterations_24, delta_l_24, marker='^', label='|Δl|_4',color="orchid")

plt.semilogy(iterations_26, delta_t_26, marker='o', label='|Δt|_4',color="slateblue")
plt.semilogy(iterations_26, delta_l_26, marker='^', label='|Δl|_4',color="slateblue")

plt.semilogy(iterations_28, delta_t_28, marker='o', label='|Δt|_4',color="cornflowerblue")
plt.semilogy(iterations_28, delta_l_28, marker='^', label='|Δl|_4',color="cornflowerblue")

# Step changes (delta)
plt.semilogy(iterations_44, delta_t_44, marker='o', label='|Δt|_4',color="dodgerblue")
plt.semilogy(iterations_44, delta_l_44, marker='^', label='|Δl|_4',color="dodgerblue")

# Step changes (delta)
plt.semilogy(iterations_46, delta_t_46, marker='o', label='|Δt|_6',color='mediumvioletred')
plt.semilogy(iterations_46, delta_l_46, marker='^', label='|Δl|_6',color='mediumvioletred')

# plt.semilogy(iterations_48, delta_t_48, marker='o', label='|Δt|', color='orange')
# plt.semilogy(iterations_48, delta_l_48, marker='^', label='|Δl|', color='orange')

plt.semilogy(iterations_64, delta_t_64, marker='o', label='|Δt|_6',color='purple')
plt.semilogy(iterations_64, delta_l_64, marker='^', label='|Δl|_6',color='purple')

plt.semilogy(iterations_66, delta_t_66, marker='o', label='|Δt|_6',color='green')
plt.semilogy(iterations_66, delta_l_66, marker='^', label='|Δl|_6',color='green')

plt.semilogy(iterations_68, delta_t_68, marker='o', label='|Δt|_6',color='red')
plt.semilogy(iterations_68, delta_l_68, marker='^', label='|Δl|_6',color='red')

plt.semilogy(iterations_84, delta_t_84, marker='o', label='|Δt|_6',color='midnightblue')
plt.semilogy(iterations_84, delta_l_84, marker='^', label='|Δl|_6',color='midnightblue')

plt.semilogy(iterations_86, delta_t_86, marker='o', label='|Δt|_6',color='darkorange')
plt.semilogy(iterations_86, delta_l_86, marker='^', label='|Δl|_6',color='darkorange')

plt.semilogy(iterations_88, delta_t_88, marker='o', label='|Δt|_6',color='maroon')
plt.semilogy(iterations_88, delta_l_88, marker='^', label='|Δl|_6',color='maroon')

plt.xlabel("Iterations", fontsize=27)
plt.ylabel("Error / Residual", fontsize=27)
plt.title("Convergence for Bèzier Segment", fontsize=27)
#plt.legend()
h1 = plt.Line2D([], [], marker='o', linestyle='-', color='black', label='|Δt|')
h2 = plt.Line2D([], [], marker='^', linestyle='-', color='black', label=r'|Δ$\lambda$|')
h12 = plt.Line2D([], [], linestyle='-', color='orchid', label='(2,4)')
h13 = plt.Line2D([], [], linestyle='-', color='slateblue', label='(2,6)')
h14 = plt.Line2D([], [], linestyle='-', color='cornflowerblue', label='(2,8)')
h3 = plt.Line2D([], [], linestyle='-', color='dodgerblue', label='(4,4)')
h4 = plt.Line2D([], [], linestyle='-', color='navy', label='(4,6)')
# h5 = plt.Line2D([], [], linestyle='-', color='orange', label='(4,8)')
h6 = plt.Line2D([], [], linestyle='-', color='purple', label='(6,4), (4,8)')
h7 = plt.Line2D([], [], linestyle='-', color='green', label='(6,6)')
h8 = plt.Line2D([], [], linestyle='-', color='red', label='(6,8)')
h9 = plt.Line2D([], [], linestyle='-', color='midnightblue', label='(8,4)')
h10 = plt.Line2D([], [], linestyle='-', color='darkorange', label='(8,6)')
h11 = plt.Line2D([], [], linestyle='-', color='maroon', label='(8,8)')
#plt.legend(handles=[h1, h2,h12,h13,h14, h3, h4, h6, h7, h8, h9, h10, h11], fontsize=14,ncol=2)
plt.grid(True, which='both', linestyle='--', alpha=0.7)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.tight_layout()
plt.show()


In [ ]:
datasets = [
    ("8_8", iterations_88, delta_t_88, delta_l_88, res_t_88, res_l_88),
    ("8_4", iterations_84, delta_t_84, delta_l_84, res_t_84, res_l_84),
    ("8_6", iterations_86, delta_t_86, delta_l_86, res_t_86, res_l_86),
    ("6_8", iterations_68, delta_t_68, delta_l_68, res_t_68, res_l_68),
    ("6_6", iterations_66, delta_t_66, delta_l_66, res_t_66, res_l_66),
    ("6_4", iterations_64, delta_t_64, delta_l_64, res_t_64, res_l_64),
    ("4_8", iterations_48, delta_t_48, delta_l_48, res_t_48, res_l_48),
    ("4_6", iterations_46, delta_t_46, delta_l_46, res_t_46, res_l_46),
    ("4_4", iterations_44, delta_t_44, delta_l_44, res_t_44, res_l_44),
    ("2_8", iterations_28, delta_t_28, delta_l_28, res_t_28, res_l_28),
    ("2_6", iterations_26, delta_t_26, delta_l_26, res_t_26, res_l_26),
    ("2_4", iterations_24, delta_t_24, delta_l_24, res_t_24, res_l_24),
    # add more datasets if needed
]
import matplotlib.cm as cm
cmap = cm.get_cmap("cool", 9)      # 9 smooth colors
colors = cmap(np.linspace(0, 1, 9))   # array of 9 RGB colors
cmap = cm.get_cmap("cool", len(datasets))
colors = cmap(np.linspace(0, 1, len(datasets)))

legend_color_handles = []
legend_labels = []

# colors = ["C0", "C1", "C2","C3", "C4", "C5", "C6", "C7", "C8"]  # one color per dataset (matplotlib cycle colors)
plt.figure(figsize=(8,6))
i=0
# Plot step changes: use same color for Δt and Δl of the same dataset
for (name, it, dt, dl, rt, rl), c in zip(datasets, colors):
    
    color = colors[i]
    plt.semilogy(it, dt, marker='o', color=color, label=f"|Δt| {name}")
    plt.semilogy(it, dl, marker='d', color=color, label=f"|Δl| {name}")
    i += 1

# # Optionally plot residuals with same color but dashed lines
# for (name, it, dt, dl, rt, rl), c in zip(datasets, colors):
#     plt.semilogy(it, rt, linestyle='--', color=c, label=f"|TS[0]| {name}")
#     plt.semilogy(it, rl, linestyle='--', color=c, label=f"|TS[1]| {name}")

plt.xlabel("Iteration", fontsize=15)
plt.ylabel("Error / Residual", fontsize=15)
plt.title("Newton Convergence", fontsize=15)
plt.legend(ncol=2)
plt.grid(True, which='both', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8,6))

# Residuals
plt.semilogy(iterations, res_t, marker='o', label='|TS[0]|', color='navy')
plt.semilogy(iterations, res_l, marker='o', label='|TS[1]|', color='navy')
# Residuals
plt.semilogy(iterations_4, res_t_4,marker='D', label='|TS[0]|_4', color='slateblue')
plt.semilogy(iterations_4, res_l_4,marker='D', label='|TS[1]|_4', color='slateblue')

# Step changes (delta)
# Residuals
plt.semilogy(iterations_6, res_t_6, marker='^', label='|TS[0]|_6',color='hotpink')
plt.semilogy(iterations_6, res_l_6, marker='^', label='|TS[1]|_6', color='hotpink')

plt.xlabel("Iteration", fontsize=15)
plt.ylabel("Error / Residual", fontsize=15)
plt.title("Newton Convergence", fontsize=15)
plt.legend()
plt.grid(True, which='both', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
datasets = [
    ("8_8", iterations, delta_t, delta_l, res_t, res_l),
    ("8_4", iterations_4, delta_t_4, delta_l_4, res_t_4, res_l_4),
    ("8_6", iterations_6, delta_t_6, delta_l_6, res_t_6, res_l_6),
    # add more datasets if needed
]

colors = ["C0", "C1", "C2"]  # one color per dataset (matplotlib cycle colors)
plt.figure(figsize=(8,6))

# Plot step changes: use same color for Δt and Δl of the same dataset
for (name, it, dt, dl, rt, rl), c in zip(datasets, colors):
    plt.semilogy(it, dt, marker='o', color=c, label=f"|Δt| {name}")
    plt.semilogy(it, dl, marker='d', color=c, label=f"|Δl| {name}")

# Optionally plot residuals with same color but dashed lines
for (name, it, dt, dl, rt, rl), c in zip(datasets, colors):
    plt.semilogy(it, rt, linestyle='--', color=c, label=f"|TS[0]| {name}")
    plt.semilogy(it, rl, linestyle='--', color=c, label=f"|TS[1]| {name}")

plt.xlabel("Iteration", fontsize=15)
plt.ylabel("Error / Residual", fontsize=15)
plt.title("Newton Convergence", fontsize=15)
plt.legend()
plt.grid(True, which='both', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
data = np.load(file / "8_8.npz")
res_t = data["res_t"]
res_l = data["res_l"]
delta_t = data["delta_t"]
delta_l = data["delta_l"]
iterations = data["iterations"]
delta_guess_sol_l = data["delta_guess_sol_l"]
delta_guess_sol_t = data["delta_guess_sol_t"]

data = np.load(file / "6_8.npz")
res_t_6 = data["res_t"]
res_l_6 = data["res_l"]
delta_t_6 = data["delta_t"]
delta_l_6 = data["delta_l"]
iterations_6 = data["iterations"]
delta_guess_sol_l_6 = data["delta_guess_sol_l"]
delta_guess_sol_t_6 = data["delta_guess_sol_t"]

data = np.load(file / "4_8.npz")
res_t_4 = data["res_t"]
res_l_4 = data["res_l"]
delta_t_4 = data["delta_t"]
delta_l_4 = data["delta_l"]
iterations_4 = data["iterations"]
delta_guess_sol_l_4 = data["delta_guess_sol_l"]
delta_guess_sol_t_4 = data["delta_guess_sol_t"]

data = np.load(file / "2_8.npz")
res_t_2 = data["res_t"]
res_l_2 = data["res_l"]
delta_t_2 = data["delta_t"]
delta_l_2 = data["delta_l"]
iterations_2 = data["iterations"]
delta_guess_sol_l_2 = data["delta_guess_sol_l"]
delta_guess_sol_t_2 = data["delta_guess_sol_t"]
# ---------------------------
# Plot
# ---------------------------
plt.figure(figsize=(8,6))

# Step changes (delta)
plt.semilogy(iterations, delta_t, marker='o', label='|Δt|', color = "navy")
plt.semilogy(iterations, delta_l, marker='x', label='|Δl|', color = "navy")
plt.semilogy(iterations, res_t, '--', label='|TS[0]|', color = "navy")
plt.semilogy(iterations, res_l, '--', label='|TS[1]|', color = "navy")

# Step changes (delta)
plt.semilogy(iterations_2, delta_t_2, marker='o', label='|Δt|_2', color="hotpink")
plt.semilogy(iterations_2, delta_l_2, marker='x', label='|Δl|_2', color="hotpink")
plt.semilogy(iterations_2, res_t_2, '--', label='|TS[0]|_2', color="hotpink")
plt.semilogy(iterations_2, res_l_2, '--', label='|TS[1]|_2', color="hotpink")

# Step changes (delta)
plt.semilogy(iterations_4, delta_t_4, marker='o', label='|Δt|_4', color="purple")
plt.semilogy(iterations_4, delta_l_4, marker='x', label='|Δl|_4', color="purple")
plt.semilogy(iterations_4, res_t_4, '--', label='|TS[0]|_4', color="purple")
plt.semilogy(iterations_4, res_l_4, '--', label='|TS[1]|_4', color="purple")

# Step changes (delta)
plt.semilogy(iterations_6, delta_t_6, marker='o', label='|Δt|_6', color = "slateblue")
plt.semilogy(iterations_6, delta_l_6, marker='x', label='|Δl|_6', color = "slateblue")
plt.semilogy(iterations_6, res_t_6, '--', label='|TS[0]|_6', color = "slateblue")
plt.semilogy(iterations_6, res_l_6, '--', label='|TS[1]|_6', color = "slateblue")

plt.xlabel("Iteration", fontsize=15)
plt.ylabel("Error / Residual", fontsize=15)
plt.title("Newton Convergence", fontsize=15)
plt.legend()
plt.grid(True, which='both', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## Newton benchmark

 8 - 4 : Use whatever nest-step we decide from newton convergence. 
Then run the drift - line, drift - halfopen, drift-bezier and compare analytical and numerical result. 
So to compare that you need the E_abs = x_num - x_sol, and E_rel = (x_num-x_abs)/x_abs. 


In [ ]:
def calculate_error(numerical, analytical):
    relE = abs(numerical - analytical)/analytical
    absE = abs(numerical - analytical)
    print(np.mean(relE))
    print(np.mean(absE))

    return relE, absE


In [ ]:
cases=100
num_iterations = 1
# res_t_finish = np.zeros(num_iterations)
# res_l_finish = np.zeros(num_iterations)
# delta_t_finish = np.zeros(num_iterations)
# delta_l_finish = np.zeros(num_iterations)
sol_t_finish = np.array([])
sol_l_finish = np.array([])
timing_taken = np.zeros(cases)
# delta_guess_sol_l_finish = np.zeros(1)
# delta_guess_sol_t_finish = np.zeros(1)

for i in range((cases)):
    check = FindRoot()
    #line_s1 = np.linspace(10, 11, cases)
    drift= tr.DriftTrajectory(s0=10., x0=12.5, xp=np.deg2rad(50.0)) 

    line_x1 = np.linspace(13.5, 15, cases)
    line = seg.LineSegment(s1=10.5, x1=line_x1[i], s2=14, x2=13.5)
    half_deg = np.linspace(41.5,44.5,cases)
    halfopen = seg.HalfOpenLineSegment(s1=10.5, x1=13, cos_t1=np.tan(np.deg2rad(half_deg[i])))
    bezier_x1 = np.linspace(13.5, 14.5, cases)
    bezier = seg.BezierSegment(s1=11, x1=bezier_x1[i], cs1=11.669, cx1=13., cs2=12., cx2=12., s2=12.669, x2=14)
    locseg= seg.segment.LocalSegment(bezier)
    loctraj= tr.trajectory.LocalTrajectory(drift)
    start = time.perf_counter()
    for j in range(500):
        check.find_crossing(seg=locseg, traj=loctraj)
    end = time.perf_counter()
    timing_taken[i] = (end - start)/500

    solution_l_data = check.solution_l
    solution_t_data = check.solution_t

    sol_t = np.array(check.solution_t)
    sol_l = np.array(check.solution_l)
    valid_sol_t  = sol_t[0]
    valid_sol_l  = sol_l[0]
    print(valid_sol_t, valid_sol_l)
    valid_sol_t   = np.where(valid_sol_t < 1e20, valid_sol_t, 1e-16)
    valid_sol_l   = np.where(valid_sol_l < 1e20, valid_sol_l, 1e-16)
    sol_t_finish = np.append(sol_t_finish, valid_sol_t)
    sol_l_finish = np.append(sol_l_finish, valid_sol_l)

In [ ]:
np.set_printoptions(precision=16)
print(np.mean(timing_taken))

In [ ]:
import xcoll as xc
filepath = xc._pkg_root.parent / "results" / "benchmarked"
segment = "bezier" 
name_numerical = f"newton_drift_{segment}_numerical.npz"
name_analytical = f"newton_drift_{segment}_analytical.npz"

In [ ]:

# Save multiple arrays in one .npz file nest_step
np.savez(filepath / name_numerical,
         sol_t=sol_t_finish,
         sol_l=sol_l_finish)

In [ ]:
data_analytical = np.load(filepath / name_analytical)
analytical_t = data_analytical["sol_t"]
analytical_l = data_analytical["sol_l"]

data_numerical = np.load(filepath / name_numerical)
numerical_t = data_numerical["sol_t"]
numerical_l = data_numerical["sol_l"]

In [ ]:
print(analytical_l[0:10])
print(numerical_l[0:10])

In [ ]:
relE_l = np.array([])
absE_l = np.array([])
relE_t = np.array([])
absE_t = np.array([])

# t 
relE_t, absE_t = calculate_error(numerical_t, analytical_t)
# l
relE_l, absE_l = calculate_error(numerical_l, analytical_l)

print(f"Mean relative error in l: {np.mean(relE_l)}")
print(f"Mean absolute error in l: {np.mean(absE_l)}")
print(f"Mean relative error in t: {np.mean(relE_t)}")
print(f"Mean absolute error in t: {np.mean(absE_t)}")

In [ ]:

print(f"Mean relative error in l: {np.mean(relE_l)}")
print(f"Mean absolute error in l: {np.mean(absE_l)}")
print(f"Mean relative error in t: {np.mean(relE_t)}")
print(f"Mean absolute error in t: {np.mean(absE_t)}")

## Simpson Convergence

In [ ]:
def calculate_error(numerical, analytical):
    relE = abs(numerical - analytical)/analytical
    absE = abs(numerical - analytical)
    print(f"relative Error: {np.mean(relE,axis=1)}")
    print(f"absolute Error: {np.mean(absE,axis=1)}")
    return relE, absE

In [ ]:
print(np.tan(np.deg2rad(180)))

In [ ]:
subintervals = np.linspace(10,1000,10) 
num_iterations = 1 
path_lengths = np.zeros((10,100)) 


timing = np.zeros((10,100)) 
import time 
for i in range(len(subintervals)): 
    for j in range(100): 
        path_sum = 0.0
        check = FindRoot() 
        drift= tr.DriftTrajectory(s0=10., x0=12.5, xp=np.deg2rad(50.0)) 
        mcs = tr.MultipleCoulombTrajectory(s0=11, x0=13, xp=np.tan(np.deg2rad(180.)), pc=1e9, 
                                           beta=1.e9/np.sqrt(0.938e9**2 + 1.e9**2), q=1, X0=0.0001, 
                                           ran_1=0.6889723883213388 , ran_2=0.4150210773485264, l1=0,l2=1) 
        line_x1 = np.linspace(13.5, 15, 100)
        line = seg.LineSegment(s1=10.5, x1=13.5, s2=14, x2=13.5) 
        locseg= seg.segment.LocalSegment(line) 
        loctraj= tr.trajectory.LocalTrajectory(mcs) 
        check.find_crossing(seg=locseg, traj=loctraj) 
        start_time = time.perf_counter() 
        for k in range(500):
            check.find_path_length(traj=loctraj, subintervals=subintervals[i]) 
            path_sum += check.path_length
        end_time = time.perf_counter() 
        timing[i,j] = ( end_time - start_time ) / 500
        path_lengths[i,j] = path_sum/500

In [ ]:
np.mean(timing, axis=1)
plt.plot(subintervals, np.mean(timing, axis=1)  )

In [ ]:
import xcoll as xc  
filepath = xc._pkg_root.parent / "results" / "simpson_bench"
#np.savez(filepath / "timing_path_length_mcs_one_500.npz", timing=timing, path_lengths=path_lengths, subintervals=subintervals)

In [ ]:
data = np.load(filepath / "timing_path_length_analytical.npz")
timing = data["timing"]
path_lengths = data["path_lengths"]
subintervals = data["subintervals"]

data = np.load(filepath / "timing_path_length_numerical.npz")
timing_num = data["timing"]
path_lengths_num = data["path_lengths"]
subintervals_num = data["subintervals"]

data = np.load(filepath / "timing_path_length_numerical_one.npz")
timing_num_one = data["timing"]
path_lengths_num_one = data["path_lengths"]
subintervals_num_one = data["subintervals"]

data = np.load(filepath / "mcs_timing_path_length_analytical.npz")
timing_mcs_analytical = data["timing"]
path_lengths_mcs = data["path_lengths"]
subintervals_mcs = data["subintervals"]

data = np.load(filepath / "mcs_timing_path_length_numerical.npz")
timing_num_mcs = data["timing"]
path_lengths_num_mcs = data["path_lengths"]
subintervals_num_mcs = data["subintervals"]

data = np.load(filepath / "mcs_timing_path_length_numerical_one.npz")
timing_num_mcs_one = data["timing"]
path_lengths_num_mcs_one = data["path_lengths"]
subintervals_num_mcs_one = data["subintervals"]

In [ ]:
data = np.load(filepath / "timing_path_length_mcs_one_500.npz")
timing_num_mcs_one = data["timing"]
path_lengths_num_mcs_one = data["path_lengths"]
subintervals_num_mcs_one = data["subintervals"]

data = np.load(filepath / "timing_path_length_drift_one_500.npz") #simpson_bench
timing_num_drift_one = data["timing"]
path_lengths_num_drift_one = data["path_lengths"]
subintervals_num_drift_one = data["subintervals"]

data = np.load(filepath / "timing_path_length_analytical_one_500.npz")
timing_num_analytical_one = data["timing"]
path_lengths_num_analytical_one = data["path_lengths"]
subintervals_num_analytical_one = data["subintervals"]

data = np.load(filepath / "timing_path_length_analytical_500.npz")
timing_num_analytical = data["timing"]
path_lengths_num_analytical = data["path_lengths"]
subintervals_num_analytical = data["subintervals"]

data = np.load(filepath / "timing_path_length_drift_500.npz") #simpson_bench
timing_num_drift = data["timing"]
path_lengths_num_drift = data["path_lengths"]
subintervals= data["subintervals"]

In [ ]:
np.set_printoptions(precision=16)
print(np.shape(path_lengths_num_mcs_one))
print(path_lengths_num_mcs_one[:,0])
for i in path_lengths_num_mcs_one[:,0]:
    print(abs(i - path_lengths_num_mcs_one[4,0]))

In [ ]:
np.set_printoptions(precision=16)
# avg_analytical = np.mean(path_lengths[:][0])
# avg_numerical = np.mean(path_lengths_num[:][0])
for i in range(len(subintervals)):
    rel_E = 0.0
    abs_E = 0.0
    for j in range(100):
        rel_E += abs(path_lengths_num_drift[i][j] - path_lengths_num_analytical[i][j]) / path_lengths_num_analytical[i][j]
        abs_E += abs(path_lengths_num_drift[i][j] - path_lengths_num_analytical[i][j])
    print(path_lengths_num_drift.shape)
    print(f"numerical path lengths: {path_lengths_num_drift[i][0]}")
    print(f"analytical path lengths: {path_lengths_num_analytical[i][0]}")
    print(f"\nSubintervals: {subintervals[i]}")
    print(f"Mean relative error: {rel_E/100}")
    print(f"Mean absolute error: {abs_E/100}")

# # ---------------------------

In [ ]:
calculate_error(path_lengths_num, path_lengths)

In [ ]:
subintervals = np.linspace(10,1000,10) 
num_iterations = 1 
path_lengths_mcs = np.zeros((10,100)) 
timing_mcs = np.zeros((10,100)) 
import time 
for i in range(len(subintervals)): 
    for j in range(100): 
        check = FindRoot() 
        drift= tr.DriftTrajectory(s0=10., x0=12.5, xp=np.deg2rad(50.0)) 
        mcs = tr.MultipleCoulombTrajectory(s0=11, x0=13, xp=np.tan(np.deg2rad(180.)), pc=1e9, 
                                           beta=1.e9/np.sqrt(0.938e9**2 + 1.e9**2), q=1, X0=0.0001, 
                                           ran_1=0.6889723883213388 , ran_2=0.4150210773485264, l1=0,l2=1) 
        line_x1 = np.linspace(13.5, 15, 100) 
        line = seg.LineSegment(s1=10.5, x1=line_x1[i], s2=14, x2=13.5) 
        locseg= seg.segment.LocalSegment(line) 
        loctraj= tr.trajectory.LocalTrajectory(mcs) 
        check.find_crossing(seg=locseg, traj=loctraj) 
        start_time = time.time() 
        check.find_path_length(traj=loctraj, subintervals=subintervals[i]) 
        end_time = time.time() 
        timing_mcs[i,j] = end_time - start_time 
        path_lengths_mcs[i,j] = check.path_length

In [ ]:
fig, ax = plt.subplots(figsize=(10,7), nrows=1, ncols=1,sharex=True)
# Step changes (delta)
plt.plot(subintervals, np.mean(timing_num_drift_one, axis=1), marker='o', label='Average CPU runtime for drift trajectory', color = "navy")
plt.plot(subintervals, np.mean(timing_num_mcs_one, axis=1), marker="o", color = "purple", alpha=1, label="Average CPU runtime for MCS trajectory")
plt.fill_between(subintervals, np.mean(timing_num_mcs_one, axis=1) + np.std(timing_num_mcs_one, axis=1), 
                     np.mean(timing_num_mcs_one, axis=1) - np.std(timing_num_mcs_one, axis=1), color = "hotpink", alpha=0.3, label="MCS trajectory std. dev.") 
plt.fill_between(subintervals, np.mean(timing_num_drift_one, axis=1) + np.std(timing_num_drift_one, axis=1), 
                     np.mean(timing_num_drift_one, axis=1) - np.std(timing_num_drift_one, axis=1), color = "dodgerblue", alpha=0.3, label="Drift trajectory std. dev.")
# ax].semilogy(subintervals, np.mean(timing, axis=1) - np.std(timing, axis=1), marker='x', color = "dodgerblue", linestyle='--')
plt.xlabel("Number of subintervals", fontsize=20)
plt.ylabel("Avg. time (s)", fontsize=25)
plt.title("Performance of Composite Simpson's Rule", fontsize=25, pad=15)
plt.axvline(x=340, color='magenta', linestyle='--', alpha=0.7, label='340 subintervals')
plt.axvline(x=615, color='magenta', linestyle='--', alpha=0.7, label='615 subintervals')

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))  # keeps first occurrence
plt.legend(by_label.values(), by_label.keys(),
           loc='upper left', fontsize=15)
plt.grid(True, which='both', linestyle='--', alpha=0.7)
plt.xticks(fontsize=14)
plt.xlim(5,1004)
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

plt.tight_layout()
# plt.suptitle("Performance of Simpson's Method", fontsize=30, y=1.07)
plt.show()


## Simpson benchmark

In [ ]:
subintervals = np.array([16, 32, 64, 128, 256, 512, 1024])
num_iterations = 1
# res_t_finish = np.zeros(num_iterations)
# res_l_finish = np.zeros(num_iterations)
# delta_t_finish = np.zeros(num_iterations)
# delta_l_finish = np.zeros(num_iterations)
sol_t_finish = np.array([])
sol_l_finish = np.array([])

# delta_guess_sol_l_finish = np.zeros(1)
# delta_guess_sol_t_finish = np.zeros(1)

for i in range((cases)):
    check = FindRoot()
    #line_s1 = np.linspace(10, 11, cases)
    drift= tr.DriftTrajectory(s0=10., x0=12.5, xp=np.deg2rad(50.0)) 

    line_x1 = np.linspace(13.5, 15, cases)
    line = seg.LineSegment(s1=10.5, x1=line_x1[i], s2=14, x2=13.5)

    locseg= seg.segment.LocalSegment(bezier)
    loctraj= tr.trajectory.LocalTrajectory(drift)
    check.find_crossing(seg=locseg, traj=loctraj)
    
    solution_l_data = check.solution_l
    solution_t_data = check.solution_t

    sol_t = np.array(check.solution_t)
    sol_l = np.array(check.solution_l)
    valid_sol_t  = sol_t[0]
    valid_sol_l  = sol_l[0]
    print(valid_sol_t, valid_sol_l)
    valid_sol_t   = np.where(valid_sol_t < 1e20, valid_sol_t, 1e-16)
    valid_sol_l   = np.where(valid_sol_l < 1e20, valid_sol_l, 1e-16)
    sol_t_finish = np.append(sol_t_finish, valid_sol_t)
    sol_l_finish = np.append(sol_l_finish, valid_sol_l)

In [ ]:
cases=100
num_iterations = 1
# res_t_finish = np.zeros(num_iterations)
# res_l_finish = np.zeros(num_iterations)
# delta_t_finish = np.zeros(num_iterations)
# delta_l_finish = np.zeros(num_iterations)
sol_t_finish = np.array([])
sol_l_finish = np.array([])

# delta_guess_sol_l_finish = np.zeros(1)
# delta_guess_sol_t_finish = np.zeros(1)
check = FindRoot()
#line_s1 = np.linspace(10, 11, cases)
drift= tr.DriftTrajectory(s0=10., x0=12.5, xp=np.deg2rad(50.0)) 
line = seg.LineSegment(s1=10.5, x1=13.5, s2=14, x2=13.5)
loctraj= tr.trajectory.LocalTrajectory(drift)
locseg= seg.segment.LocalSegment(line)
check.find_crossing(seg=locseg, traj=loctraj) 

check.find_path_length(traj=loctraj)

pathlength = check.path_length
print(pathlength)

In [ ]:
cases=100
num_iterations = 1
# res_t_finish = np.zeros(num_iterations)
# res_l_finish = np.zeros(num_iterations)
# delta_t_finish = np.zeros(num_iterations)
# delta_l_finish = np.zeros(num_iterations)
sol_t_finish = np.array([])
sol_l_finish = np.array([])

# delta_guess_sol_l_finish = np.zeros(1)
# delta_guess_sol_t_finish = np.zeros(1)
check = FindRoot()
#line_s1 = np.linspace(10, 11, cases)
drift= tr.DriftTrajectory(s0=10., x0=12.5, xp=np.deg2rad(50.0)) 
line = seg.LineSegment(s1=10.5, x1=13.5, s2=14, x2=13.5)
loctraj= tr.trajectory.LocalTrajectory(drift)
locseg= seg.segment.LocalSegment(line)
check.find_crossing(seg=locseg, traj=loctraj) 

check.find_path_length(traj=loctraj)


In [ ]:
#numerical
pathlength = check.path_length
print(pathlength)

In [ ]:
#analytical
pathlength = check.path_length
print(pathlength)

In [ ]:
3 % 2